In [12]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [13]:
from my_functions import *

In [14]:
time_column='datetime'
feature_columns = ['station_id', 'month', 'day', 'hour',
       'lag_4h_percentage_docks_available',
       'lag_3h_percentage_docks_available',
       'lag_2h_percentage_docks_available',
       'lag_1h_percentage_docks_available', 'capacity', 'lat', 'lon',
       'altitude', 'post_code', 'year', 'day_of_week',
       'lag_1h_is_interpolated', 'lag_2h_is_interpolated',
       'lag_3h_is_interpolated', 'lag_4h_is_interpolated', 'FCB_Location',
       'FCB_Score', 'wea_prep', 'wea_sun', 'wea_temp', 'wea_wind_sp',
       'holiday', 'd_lag_1d_holiday',
       'd_lead_1d_holiday', 't_lag_1h_FCB_Location', 't_lag_1h_FCB_Score', 't_lag_1h_wea_prep',
       't_lag_1h_wea_sun', 't_lag_1h_wea_temp', 't_lag_1h_wea_wind_sp',
       't_lag_2h_FCB_Location', 't_lag_2h_FCB_Score', 't_lag_2h_wea_prep',
       't_lag_2h_wea_sun', 't_lag_2h_wea_temp', 't_lag_2h_wea_wind_sp',
       't_lag_3h_FCB_Location', 't_lag_3h_FCB_Score', 't_lag_3h_wea_prep',
       't_lag_3h_wea_sun', 't_lag_3h_wea_temp', 't_lag_3h_wea_wind_sp',
       't_lag_4h_FCB_Location', 't_lag_4h_FCB_Score', 't_lag_4h_wea_prep',
       't_lag_4h_wea_sun', 't_lag_4h_wea_temp', 't_lag_4h_wea_wind_sp',
       't_lead_1h_FCB_Location', 't_lead_1h_FCB_Score', 't_lead_1h_wea_prep',
       't_lead_1h_wea_sun', 't_lead_1h_wea_temp', 't_lead_1h_wea_wind_sp',
       't_lead_2h_FCB_Location', 't_lead_2h_FCB_Score', 't_lead_2h_wea_prep',
       't_lead_2h_wea_sun', 't_lead_2h_wea_temp', 't_lead_2h_wea_wind_sp',
       't_lead_3h_FCB_Location', 't_lead_3h_FCB_Score', 't_lead_3h_wea_prep',
       't_lead_3h_wea_sun', 't_lead_3h_wea_temp', 't_lead_3h_wea_wind_sp',
       't_lead_4h_FCB_Location', 't_lead_4h_FCB_Score', 't_lead_4h_wea_prep',
       't_lead_4h_wea_sun', 't_lead_4h_wea_temp', 't_lead_4h_wea_wind_sp']
target_column = 'percentage_docks_available'

In [15]:
# Combine all columns into one list
selected_columns = feature_columns + [time_column] + [target_column]


In [16]:
# Load only the specified columns
df = pd.read_csv('data/Train_prepared_v3.csv') #.sample(frac=frac_load, random_state=42)

C:\Users\ignas\AppData\Local\Temp\ipykernel_21744\3046940161.py:2: DtypeWarning: Columns (17,19,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Train_prepared_v3.csv') #.sample(frac=frac_load, random_state=42)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14355983 entries, 0 to 14355982
Data columns (total 81 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Unnamed: 0                         int64  
 1   station_id                         int64  
 2   year                               int64  
 3   month                              int64  
 4   day                                int64  
 5   hour                               int64  
 6   date                               object 
 7   num_docks_available                float64
 8   datetime                           object 
 9   capacity                           int64  
 10  lat                                float64
 11  lon                                float64
 12  altitude                           float64
 13  post_code                          float64
 14  percentage_docks_available         float64
 15  day_of_week                        int64  
 16  lag_1h_percentag

In [18]:
# Convert last_reported to datetime
df[time_column] = pd.to_datetime(df[time_column])

In [19]:
convert_object_columns_to_boolean(df)


convert_object_columns_to_category(df)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14355983 entries, 0 to 14355982
Data columns (total 81 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   Unnamed: 0                         int64         
 1   station_id                         int64         
 2   year                               int64         
 3   month                              int64         
 4   day                                int64         
 5   hour                               int64         
 6   date                               category      
 7   num_docks_available                float64       
 8   datetime                           datetime64[ns]
 9   capacity                           int64         
 10  lat                                float64       
 11  lon                                float64       
 12  altitude                           float64       
 13  post_code                          float64       
 14  

## Model

In [21]:
# Define train, validation, and test splits based on time
# Using 70% for training, 15% for validation, and 15% for testing
train_end = df[time_column].quantile(0.7)

train_data = df[df[time_column] <= train_end]
val_data = df[df[time_column] > train_end]

import gc

del df  # Delete the DataFrame
gc.collect()  # Force garbage collection

# Split features and target
X_train = train_data[feature_columns]
y_train = train_data[target_column]
X_val = val_data[feature_columns]
y_val = val_data[target_column]


In [22]:
#  Imports for modeling
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
# import optuna


In [23]:

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=True)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data, free_raw_data=True)


()'learning_rate': 0.017217810024463404, 'num_leaves': 592, 'max_depth': 35, 'min_data_in_leaf': 62, 'feature_fraction': 0.9939668052243837, 'bagging_fraction': 0.8406963017607484, 'bagging_freq': 5}. Best is trial 11 with value: 0.09624436082560552.
Training until validation scores don't improve for 100 rounds

In [24]:
# import lightgbm as lgb
# import optuna
# from sklearn.metrics import mean_squared_error

# # Define the objective function for Optuna to optimize
# def objective(trial):
#     # Define the hyperparameter search space using Optuna
#     params = {
#         'objective': 'regression',  # Task type (regression)
#         'metric': 'rmse',  # Evaluation metric (Root Mean Squared Error)
#         'verbosity': -1,  # Suppress LightGBM logs
#         'boosting_type': 'gbdt',  # Gradient Boosting Decision Trees
#         'feature_pre_filter': False,  # No pre-filtering of features
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),  # Learning rate (between 0.01 and 0.3)
#         'num_leaves': trial.suggest_int('num_leaves', 20, 1000),  # Maximum number of leaves per tree
#         'max_depth': trial.suggest_int('max_depth', 3, 100),  # Maximum tree depth
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),  # Minimum number of data in a leaf
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.8, 1.0),  # Fraction of features to use
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.8, 1.0),  # Fraction of data to use for bagging
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),  # Frequency of bagging
#     }

#     # Train the model with the given hyperparameters
#     model = lgb.train(
#         params, 
#         train_data,  # Training dataset
#         valid_sets=[val_data],  # Validation dataset
#         num_boost_round=1000,  # Maximum training rounds
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=100),  # Stop early if no improvement
#             lgb.log_evaluation(0)  # Log evaluation results
#         ]
#     )

#     # Make predictions on the validation set
#     preds = model.predict(X_val)

#     # Calculate RMSE (Root Mean Squared Error)
#     rmse = mean_squared_error(y_val, preds, squared=False)

#     return rmse  # Return RMSE as the metric to minimize

# # ========================
# # 2. Run Hyperparameter Tuning
# # ========================

# # Create an Optuna study to minimize the RMSE
# study = optuna.create_study(direction='minimize')

# # Run optimization for 10 trials
# study.optimize(objective, n_trials=100)

# # Retrieve the best hyperparameters from the study
# best_params = study.best_trial.params
# print("Best Hyperparameters:", best_params)

# # ========================
# # 3. Train Final Model with Best Hyperparameters
# # ========================

# # Use the best parameters found during tuning
# final_params = {
#     'objective': 'regression',
#     'metric': 'rmse',
#     'verbosity': -1,
#     'boosting_type': 'gbdt',
#     **best_params  # Merge best hyperparameters into final parameters
# }


In [25]:
final_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.035304947581187546, 'num_leaves': 403, 'max_depth': 43, 'min_data_in_leaf': 96, 'feature_fraction': 0.9000738036420937, 'bagging_fraction': 0.8577277335249167, 'bagging_freq': 1
}

In [26]:



# Train the final model with the best hyperparameters
model = lgb.train(
    final_params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=5000,  # Maximum training rounds
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),  # Early stopping
        lgb.log_evaluation(50)  # Log evaluation results every 50 iterations
    ]
)

# Save the trained model to a file
model.save_model('lightgbm_free_docks_model_2.txt')


Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 0.111541
[100]	valid_0's rmse: 0.101087
[150]	valid_0's rmse: 0.0992409
[200]	valid_0's rmse: 0.0982039
[250]	valid_0's rmse: 0.0976251
[300]	valid_0's rmse: 0.0972987
[350]	valid_0's rmse: 0.0970501
[400]	valid_0's rmse: 0.0968937
[450]	valid_0's rmse: 0.0967823
[500]	valid_0's rmse: 0.0966821
[550]	valid_0's rmse: 0.0966066
[600]	valid_0's rmse: 0.096498
[650]	valid_0's rmse: 0.0964259
[700]	valid_0's rmse: 0.0963773
[750]	valid_0's rmse: 0.0963174
[800]	valid_0's rmse: 0.096266
[850]	valid_0's rmse: 0.0962068
[900]	valid_0's rmse: 0.096173
[950]	valid_0's rmse: 0.0961369
[1000]	valid_0's rmse: 0.0961062
[1050]	valid_0's rmse: 0.0960743
[1100]	valid_0's rmse: 0.0960394
[1150]	valid_0's rmse: 0.0960072
[1200]	valid_0's rmse: 0.0959845
[1250]	valid_0's rmse: 0.0959572
[1300]	valid_0's rmse: 0.0959397
[1350]	valid_0's rmse: 0.0959153
[1400]	valid_0's rmse: 0.0958995
[1450]	valid_0's rmse: 0.0958933
[1500]

In [27]:

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importance(importance_type='gain')
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))


Feature Importance:
                              feature    importance
7   lag_1h_percentage_docks_available  7.339136e+06
6   lag_2h_percentage_docks_available  3.455162e+05
3                                hour  5.644933e+04
5   lag_3h_percentage_docks_available  5.243729e+04
11                           altitude  5.032144e+04
..                                ...           ...
40              t_lag_3h_FCB_Location  1.665115e+01
29                 t_lag_1h_FCB_Score  1.368881e+01
41                 t_lag_3h_FCB_Score  9.228150e+00
47                 t_lag_4h_FCB_Score  6.055074e+00
46              t_lag_4h_FCB_Location  5.526932e+00

[76 rows x 2 columns]


In [37]:
feature_importance.sort_values('importance', ascending=False).to_csv(f'model_3/feature_importance.csv', index=False)


In [29]:
# Train final model on combined train and validation data using best iteration
final_model = lgb.train(
    final_params,
    lgb.Dataset(pd.concat([X_train, X_val]), label=pd.concat([y_train, y_val])),
    num_boost_round=3052 #model.best_iteration  # Use best iteration from previous validation
)

# Save the final model trained on combined data
final_model.save_model('lightgbm_free_docks_model_final_2.txt')


In [30]:
sample= pd.read_csv('data\Metadata_prepared_v2.csv')

In [31]:
convert_object_columns_to_boolean(sample)
convert_object_columns_to_category(sample)

In [32]:
# Reorder the columns of prediction_data to match X_train's column order
prediction_data = sample[X_train.columns]


# Check if the columns of prediction_data match the columns of X_train
columns_match = prediction_data.columns.tolist() == X_train.columns.tolist()

# Print if the columns match
print("Columns match:", columns_match)

# If the columns don't match, print the differences
if not columns_match:
    print("Columns in prediction_data that are not in X_train:", set(prediction_data.columns) - set(X_train.columns))
    print("Columns in X_train that are not in prediction_data:", set(X_train.columns) - set(prediction_data.columns))

Columns match: True


In [33]:
# Make predictions
predictions = final_model.predict(prediction_data)

# Print or further process predictions
print("Predictions:", predictions)

Predictions: [0.30474153 0.47701968 0.74546052 ... 0.51384785 0.51078024 0.52531953]


In [34]:
# Create final submission dataframe with explicit index
final_submission = pd.DataFrame({
    'index': range(len(predictions)),
    'percentage_docks_available': predictions
})

In [35]:
# Create the directory if it doesn't exist
import os
os.makedirs('model_3', exist_ok=True)

# Get existing files matching pattern
import glob
import re

existing_files = glob.glob('model_3/Sub_model_3_plain_*.csv')

# Extract existing numbers and find max
numbers = [int(re.search(r'plain_(\d+)', f).group(1)) for f in existing_files if re.search(r'plain_(\d+)', f)]
next_num = max(numbers) + 1 if numbers else 1

# Create final submission dataframe with explicit index and convert to numpy array first
submission_data = np.column_stack((np.arange(len(predictions)), predictions))
final_submission = pd.DataFrame(
    submission_data, 
    columns=['index', 'percentage_docks_available']
).astype({'index': int, 'percentage_docks_available': float})

# Save to CSV
output_file = f'model_3/Sub_model_3_plain_{next_num}.csv'
print(f"\nSaving final submission to: {output_file}")

# Use numpy savetxt instead of pandas to_csv
np.savetxt(
    output_file,
    submission_data,
    delimiter=',',
    header='index,percentage_docks_available',
    comments='',
    fmt=['%d', '%.6f']
)


Saving final submission to: model_3/Sub_model_3_plain_3.csv


In [36]:
print(type(final_submission))  # Confirm it's a DataFrame
print(final_submission.dtypes)  # Check column data types
print(final_submission.columns)  # See column names
print(final_submission.index)  # Check index structure
print(final_submission.head())  # Preview data





<class 'pandas.core.frame.DataFrame'>
index                           int32
percentage_docks_available    float64
dtype: object
Index(['index', 'percentage_docks_available'], dtype='object')
RangeIndex(start=0, stop=401511, step=1)
   index  percentage_docks_available
0      0                    0.304742
1      1                    0.477020
2      2                    0.745461
3      3                    0.787848
4      4                    0.763783


------------------------
-------------------------
